# 전체 범위의 어텐션 스코어 저장 TO CSV (가감된 어텐션)
TP, TN, FP, FN 4가지 상황에서 모든 패치의 어텐션을 CSV 파일로 저장합니다.

14*14크기의 어텐션 맵에서 index는 가장 위쪽이 x가 0, 가장 왼쪽이 y가 0입니다.

### Import Libraries

In [1]:
import torch
import numpy as np
import pickle
import matplotlib.pyplot as plt
import glob
import gc
import os
import time
import pandas as pd

from vit_rollout import VITAttentionRollout

### Definition

In [2]:
def custom_attention_mask(label, mask):
    custom_mask = mask

    if label == 'TP':
        # Edit attentions
        for x in range(0, 13+1): # 차선 밖
            for y in range(0, 3+1):
                custom_mask[x][y] = custom_mask[x][y] * 0.5
            for y in range(9, 13+1):
                custom_mask[x][y] = custom_mask[x][y] * 0.5
        
        for x in range(0, 13+1): # 반대편 차선 가드레일
            custom_mask[x][4] = custom_mask[x][4] * 0.35
        
        for x in range(0, 13+1): # 반대편 차선
            custom_mask[x][5] = custom_mask[x][5] * 0.5

        for x in range(8, 13+1): # Ego Vehicle 뒤
            for y in range(6, 8+1):
                custom_mask[x][y] = custom_mask[x][y] * 0.5

        # 특이 지점
        custom_mask[6][7] = custom_mask[6][7] * 0.3
        custom_mask[7][6] = custom_mask[7][6] * 0.3
        custom_mask[8][6] = custom_mask[8][6] * 0.3
        custom_mask[9][6] = custom_mask[9][6] * 0.3


    if label == 'TN':
        # Edit attention
        for x in range(0, 13+1): # 반대편 차선
            for y in range(0, 4+1):
                custom_mask[x][y] = custom_mask[x][y] * 0.5
        
        for x in range(0, 13+1): # 중앙분리대
            custom_mask[x][5] = custom_mask[x][5] * 0.3

        custom_mask[9][7] = custom_mask[9][7] * 0.05 # 특이지점

        for x in range(7, 13+1): # Ego vehicle 뒤 차도
            for y in range(6, 8+1):
                custom_mask[x][y] = custom_mask[x][y] * 0.3

        for x in range(0, 13+1): # 차도 밖
            for y in range(9, 13+1):
                custom_mask[x][y] = custom_mask[x][y] * 0.6


    if label == 'FP':
        # Edit attention
        custom_mask[5][4] = custom_mask[5][4] * 0.2 # 특이지점


    if label == 'FN':
        # Edit attention
        for x in range(0, 13+1): # 차도밖
            for y in range(0, 3+1):
                custom_mask[x][y] = custom_mask[x][y] * 0.15 
            for y in range(9, 13+1):
                custom_mask[x][y] = custom_mask[x][y] * 0.15

        for x in range(0, 13+1): # 반대편 차선
            for y in range(4, 5+1):
                custom_mask[x][y] = custom_mask[x][y] * 0.2
        
        for x in range(8, 13+1): #Ego Vehicle 뒤
            for y in range(6, 7+1):
                custom_mask[x][y] = custom_mask[x][y] * 0.5


    return custom_mask

In [3]:
def save_attention_score_csv(model, PATH_data_list, label):
    for PATH_data in PATH_data_list:
        data_number = PATH_data.split('/')[-1].split('.')[0].split('\\')[-1]
        
        # Load data: using CPU
        with open(PATH_data, 'rb') as f:
            data = pickle.load(f)

        input_tensor = data['tensor']
        input_tensor = input_tensor.reshape((1,14,28,28))
        input_tensor = input_tensor.to(torch.float32).cpu()

        # Rollout attentions
        attention_rollout = VITAttentionRollout(model, head_fusion='max', discard_ratio=0.90)
        mask_law = attention_rollout(input_tensor) # (14, 14)

        df = pd.DataFrame(custom_attention_mask(label=label, mask=mask_law))
        df.to_csv("attention_score/"+label+"/"+data_number+'.csv')

### Load Data Path

In [4]:
# 285~325
PATH_sample_data_TP_list = glob.glob("../../dataset/data_preprocessed/Vulner/21-12-01-11-41-59_end_extract_drive8/*")

# 300~350
PATH_sample_data_TN_list = glob.glob("../../dataset/data_preprocessed/None-crash/21-12-01-11-07-44_end_extract_drive3/*")

# 95~115
PATH_sample_data_FP_list = glob.glob("../../dataset/data_preprocessed/None-crash/21-12-29-11-04-19_end_extract_drive14/*") 

# 290~305 -> start with 292
PATH_sample_data_FN_list = glob.glob("../../dataset/data_preprocessed/Vulner/21-12-14-15-04-44_end_extract_drive2/*") 

PATH_sample_data_TN_list = PATH_sample_data_TN_list[297:347+1]
PATH_sample_data_FP_list = PATH_sample_data_FP_list[92:112+1]

model = torch.load('../../model/model.pt').cpu()

In [5]:
os.makedirs('attention_score/TP', exist_ok=True)
os.makedirs('attention_score/TN', exist_ok=True)
os.makedirs('attention_score/FP', exist_ok=True)
os.makedirs('attention_score/FN', exist_ok=True)

### Save attention score to csv

In [6]:
save_attention_score_csv(model=model, PATH_data_list=PATH_sample_data_TP_list, label='TP')
save_attention_score_csv(model=model, PATH_data_list=PATH_sample_data_TN_list, label='TN')
save_attention_score_csv(model=model, PATH_data_list=PATH_sample_data_FP_list, label='FP')
save_attention_score_csv(model=model, PATH_data_list=PATH_sample_data_FN_list, label='FN')